### 의도 분류 테스트

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Executing video content`, `daily information`, 'Performing a specific task', or `Other`.

Do not respond with more than one sentence.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model="gpt-3.5-turbo-0125")
    | StrOutputParser()
)

# chain.invoke({"question": "Play the movie Titanic."})

chain.invoke({"question": "What is the temperature in Seoul today? Please tell me the temperature in Celsius."})




'Daily information'

In [5]:
video_content_chain = PromptTemplate.from_template(
    """You are an Agent with the ability to play videos mapped to titles based on embedded video information. \
It doesn't actually play the video, but if the user asks you to do so by mentioning a specific movie title, respond that you will play the video for that title. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")


daily_information_chain = PromptTemplate.from_template(
    """You are an Agent who is familiar with daily life information. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")


general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
def route(info):
    if "video" in info["topic"].lower():
        return video_content_chain
    elif "daily information" in info["topic"].lower():
        return daily_information_chain
    else:
        return general_chain

In [7]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

In [8]:
full_chain.invoke({"question": "Play the movie Titanic."})

AIMessage(content='I will play the video for Titanic.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 76, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b82d8dbc-941a-448a-8459-a86776e9b8bb-0')

In [10]:
full_chain.invoke({"question": "What is the temperature in Seoul today? Please answer in degrees Celsius and include today's date information."})

AIMessage(content="As of today, the temperature in Seoul is 15 degrees Celsius. Today's date is October 15th.", response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 49, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1e7da6a7-b860-4732-b334-b56b7e2bf591-0')